In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model

In [5]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [6]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [7]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 03s]
val_accuracy: 0.8821666836738586

Best val_accuracy So Far: 0.893916666507721
Total elapsed time: 00h 10m 41s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



In [8]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4979 - accuracy: 0.8237 - val_loss: 0.3877 - val_accuracy: 0.8592
Epoch 2/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3729 - accuracy: 0.8635 - val_loss: 0.3628 - val_accuracy: 0.8716
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3302 - accuracy: 0.8792 - val_loss: 0.3456 - val_accuracy: 0.8737
Epoch 4/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3049 - accuracy: 0.8880 - val_loss: 0.3565 - val_accuracy: 0.8718
Epoch 5/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2873 - accuracy: 0.8938 - val_loss: 0.3197 - val_accuracy: 0.8832
Epoch 6/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2698 - accuracy: 0.8998 - val_loss: 0.3363 - val_accuracy: 0.8833
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2565 - accuracy: 0.9047 - val_loss: 0.3129 - val_accuracy:

In [9]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/49
1500/1500 [==============================] - 7s 4ms/step - loss: 0.4917 - accuracy: 0.8253 - val_loss: 0.3950 - val_accuracy: 0.8558
Epoch 2/49
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3670 - accuracy: 0.8669 - val_loss: 0.3834 - val_accuracy: 0.8628
Epoch 3/49
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3312 - accuracy: 0.8774 - val_loss: 0.3434 - val_accuracy: 0.8752
Epoch 4/49
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3052 - accuracy: 0.8860 - val_loss: 0.3308 - val_accuracy: 0.8822
Epoch 5/49
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2875 - accuracy: 0.8948 - val_loss: 0.3206 - val_accuracy: 0.8864
Epoch 6/49
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2725 - accuracy: 0.8985 - val_loss: 0.3896 - val_accuracy: 0.8587
Epoch 7/49
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2573 - accuracy: 0.9028 - val_loss: 0.3194 - val_accuracy:

In [10]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.5891 - accuracy: 0.8864
[test loss, test accuracy]: [0.5891376733779907, 0.8863999843597412]
